__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/pandasklar/blob/master/jupyter/23_Let_DataFrames_Interact.ipynb)__

# Let DataFrames Interact
* `isin`: isin over several columns. Returns a mask for df1: The rows of df1 that match the ones in df2 in the specified columns.
* `update_col`:     Transfers one column of data from one dataframe to another dataframe.<br>
   Unlike a simple merge, the index and the dtypes are retained. Handles dups and conditions. Verbose if wanted.

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Dropbox
environment['lib_path']     = /home/me/Dropbox/31_Projekte/01_Python/libs
Start Time: 10:43:45


In [2]:
import pandas     as pd 
import bpyth      as bpy

# pandasklar
try:
    import pandasklar as pak 
except ImportError as e:
    !pip install pandasklar
    import pandasklar as pak   
    
# verbose
pak.Config.set('VERBOSE', True)

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions



## isin
over several columns

In [3]:
?pak.isin

Signature: pak.isin(df1, df2, on=[], left_on=[], right_on=[])
Docstring:
isin over several columns. 
Returns a mask for df1: The rows of df1 that match the ones in df2 in the specified columns.
File:      ~/mambaforge/envs/python_06/lib/python3.9/site-packages/pandasklar/pandas.py
Type:      function

In [4]:
df1 = pak.dataframe( [ list('Zugzwang'),  # 'Zu'  
                       list('Wurmloch'), 
                       list('Witzbold'), 
                       list('Heuernte'), 
                       list('Halbgott'),  # 'Ha' 
                       list('Halbzeit'),  # 'Ha' 
                       list('Hausfrau'),  # 'Ha'       
                       list('Hebelarm'),                     
                ] )

df2 = pak.dataframe( [ list('Babykorb'), 
                       list('abfällig'), 
                       list('Abgründe'), 
                       list('Kätzchen'), 
                       list('Landwirt'), 
                       list('Handkuss'),  # 'Ha'
                       list('Saugrohr'),       
                       list('Zuzügler'),  # 'Zu'                   
                ] )

df2

Input rtype=('list', 'list', 'str') shape=(8, 8)
rotated=False Output rtype=('DataFrame', 'Series') shape=(8, 8)
Input rtype=('list', 'list', 'str') shape=(8, 8)
rotated=False Output rtype=('DataFrame', 'Series') shape=(8, 8)


,A,B,C,D,E,F,G,H
0,B,a,b,y,k,o,r,b
1,a,b,f,ä,l,l,i,g
2,A,b,g,r,ü,n,d,e
3,K,ä,t,z,c,h,e,n
4,L,a,n,d,w,i,r,t
5,H,a,n,d,k,u,s,s
6,S,a,u,g,r,o,h,r
7,Z,u,z,ü,g,l,e,r


In [5]:
mask = pak.isin(df1, df2, on=['A','B'])
df1[mask]

,A,B,C,D,E,F,G,H
0,Z,u,g,z,w,a,n,g
4,H,a,l,b,g,o,t,t
5,H,a,l,b,z,e,i,t
6,H,a,u,s,f,r,a,u


In [6]:
mask = ~pak.isin(df1, df2, on=['A','B'])
df1[mask]

,A,B,C,D,E,F,G,H
1,W,u,r,m,l,o,c,h
2,W,i,t,z,b,o,l,d
3,H,e,u,e,r,n,t,e
7,H,e,b,e,l,a,r,m


## update_col

In [7]:
?pak.update_col

Signature:
pak.update_col(
    df_to,
    df_from,
    on=[],
    left_on=[],
    right_on=[],
    col='',
    col_rename='',
    col_score='',
    func='',
    cond='',
    keep='',
    return_mask=False,
    verbose=None,
)
Docstring:
Transfers one column of data from one dataframe to another dataframe.
Unlike a simple merge, the index and the dtypes are retained. 
Handles dups and conditions. Verbose if wanted.

df_to:        Dataframe to be changed
df_from:      Dataframe that contains the new data. Does not have to be duplicate-free.
on:           Column name or list of column names whose values must match 
left_on:      Different for left and right if necessary
right_on:     Different for left and right if necessary    
col:          Name of the column to be transmitted. Must exist in df_from. 
              If not present in df_to, it will be appended. 
              If already present, matching values will be overwritten.
col_rename:   New name for col, if specified. If already

In [8]:
# create test data
df = pak.people()
df = pak.drop_cols(df,['history'])
df

,first_name,age,age_class,postal_code,birthplace,secret,features
0,Erwin,35,30,84566,Berlin,dÄ2h2wÜuc,"{D, S}"
1,Klaus,26,20,13370,Bremen,83D4dV7,{G}
2,Tanja,36,30,36443,<NA>,YIYo0KüMNo,"{J, 1}"
3,Lennard,30,30,34447,Bremen,JuUmüXh,"{x, m, g}"
4,Martha,27,20,44994,Bremen,kzutfKV,"{R, J}"
...,...,...,...,...,...,...,...
95,Yvonne,27,20,40992,Berlin,mULcX9D,{0}
96,Torsten,29,20,53506,Berlin,jgUSPYunÄX,"{D, O, n}"
97,Friedrich,25,20,99261,<NA>,saÜdJFn,{Q}
98,Torsten,31,30,94048,<NA>,YewSWlPN,"{t, u, Y}"


In [9]:
# create df_from
# we want to transfer data from df_from to df
df_from = df.head(20).copy()
for c in ['birthplace', 'features']:
    df_from[c] = '##' + df_from[c].astype(str) + '##'
for c in ['age','age_class','postal_code']:
    df_from[c] = -df_from[c]    
df_from.head(3)

,first_name,age,age_class,postal_code,birthplace,secret,features
0,Erwin,-35,-30,-84566,##Berlin##,dÄ2h2wÜuc,"##{'D', 'S'}##"
1,Klaus,-26,-20,-13370,##Bremen##,83D4dV7,##{'G'}##
2,Tanja,-36,-30,-36443,##<NA>##,YIYo0KüMNo,"##{'J', '1'}##"


In [10]:
# Transfer features data. Use first_name and secret as idenitfier.
df = pak.update_col(df, df_from, on=['first_name','secret'], col='features')
df

update_col: 20 cells written into existing column


,first_name,age,age_class,postal_code,birthplace,secret,features
0,Erwin,35,30,84566,Berlin,dÄ2h2wÜuc,"##{'D', 'S'}##"
1,Klaus,26,20,13370,Bremen,83D4dV7,##{'G'}##
2,Tanja,36,30,36443,<NA>,YIYo0KüMNo,"##{'J', '1'}##"
3,Lennard,30,30,34447,Bremen,JuUmüXh,"##{'x', 'm', 'g'}##"
4,Martha,27,20,44994,Bremen,kzutfKV,"##{'R', 'J'}##"
...,...,...,...,...,...,...,...
95,Yvonne,27,20,40992,Berlin,mULcX9D,{0}
96,Torsten,29,20,53506,Berlin,jgUSPYunÄX,"{D, O, n}"
97,Friedrich,25,20,99261,<NA>,saÜdJFn,{Q}
98,Torsten,31,30,94048,<NA>,YewSWlPN,"{t, u, Y}"


In [11]:
# Transfer age data. Use age_class as idenitfier.
df = pak.update_col(df, df_from, on=['first_name','secret'], col='age', keep='_old')

update_col: 20 cells written into existing column


In [12]:
# have a look
mask = df.age < 0
a = df[ mask].head(10)
b = df[~mask].head(10)
pd.concat([a,b])

,first_name,age,age_class,postal_code,birthplace,secret,features,age_old
0,Erwin,-35,30,84566,Berlin,dÄ2h2wÜuc,"##{'D', 'S'}##",35
1,Klaus,-26,20,13370,Bremen,83D4dV7,##{'G'}##,26
2,Tanja,-36,30,36443,<NA>,YIYo0KüMNo,"##{'J', '1'}##",36
3,Lennard,-30,30,34447,Bremen,JuUmüXh,"##{'x', 'm', 'g'}##",30
4,Martha,-27,20,44994,Bremen,kzutfKV,"##{'R', 'J'}##",27
5,Adolf,-33,30,58563,Berlin,uPVqX,##{'0'}##,33
6,Tobias,-39,30,95158,Bremen,äsxyI38,##{'Q'}##,39
7,Lukas,-30,30,50919,Berlin,cNGXo1U,"##{'k', 'F', 'c', 'H'}##",30
8,Lisa,-22,20,51959,Berlin,1K4WrGäJN,##set()##,22
9,Carl,-30,30,<NA>,Berlin,U6NüZaDrM,"##{'m', 'z', 'a'}##",30
